In [ ]:
# default_exp models.MultiInputNet

# MultiInputNet

> This is an implementation created by Ignacio Oguiza - timeseriesAI@gmail.com. It can be used to combine different types of deep learning models into a single one that will accept multiple inputs from a MixedDataLoaders.

In [ ]:
#export
from tsai.imports import *
from tsai.models.layers import *
from tsai.models.utils import *

In [ ]:
#export
class MultiInputNet(Module):
    
    def __init__(self, *models, c_out=None, reshape_fn=None, multi_output=False, custom_head=None, device=None, **kwargs):
        r"""
        Args:
            models       : list of models (one model per dataloader in dls). They all must have a head.
            c_out        : output layer size.
            reshape_fn   : callable to transform a 3d input into a 2d input (Noop, Reshape(-1), GAP1d())
            multi_output : determines if the model creates M+1 output (one per model plus a combined one), or just a single output (combined one).
            custom_head  : allows you to pass a custom joint head. If None a MLP will be created (you can pass 'layers' to this default head using kwargs)
            device       : cpu or cuda. If None, default_device() will be chosen.
            kwargs       : head kwargs
        """

        c_out = ifnone(c_out, get_layers(models[0], cond=is_linear)[-1].out_features)
        self.M = len(models)
        self.m = []
        self.backbones = nn.ModuleList()
        self.heads = nn.ModuleList()
        head_nf = 0
        min_nf = np.inf
        for i, model in enumerate(models):
            try: # if subscriptable
                self.heads.append(model[1])
                self.backbones.append(model[0])
            except:
                self.heads.append(model.head)
                model.head = Identity()
                self.backbones.append(model)
            self.m.append(Sequential(self.backbones[-1], self.heads[-1]))
            head_nf += model.head_nf
            min_nf = min(min_nf, model.head_nf)

        self.head_nf = head_nf
        if custom_head is None: head = create_fc_head(head_nf, c_out, 1, **kwargs)
        else: head = custom_head(self.head_nf, c_out, **kwargs)
        self.heads.append(head)
        self.multi_output = multi_output
        self.m.append(self)
        self.reshape = ifnone(reshape_fn, GAP1d())
        self.concat = Concat(dim=1)
        device = ifnone(device, default_device())
        self.to(device=device)

    def forward(self, xs):
        xs = tuple(*xs) if len(xs) == 1 else xs
        out = []
        for k in range(self.M):
            x = xs[k]
            # Create separate features
            feat = self.backbones[k](*x) if isinstance(x, (list, tuple, L)) else self.backbones[k](x)

            # Process features separately
            if self.training and self.multi_output: out.append(self.heads[k](feat))
            
            # Concat features
            if feat.ndim == 3: feat = self.reshape(feat)
            concat_feats = feat if k==0 else self.concat([concat_feats, feat])
            
        # Process joint features
        out.append(self.heads[-1](concat_feats))
        if self.training and self.multi_output: return out
        else:  return out[0]

In [ ]:
from fastai.data.transforms import *
from tsai.data.all import *
from tsai.models.utils import *
from tsai.models.InceptionTimePlus import *
from tsai.models.TabModel import *

In [ ]:
dsid = 'NATOPS'
X, y, splits = get_UCR_data(dsid, split_data=False)
ts_features_df = get_ts_features(X, y)

Feature Extraction: 100%|██████████| 40/40 [00:03<00:00, 10.85it/s]


In [ ]:
# raw ts
tfms  = [None, [Categorize()]]
batch_tfms = TSStandardize()
ts_dls = get_ts_dls(X, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms)
ts_model = build_ts_model(InceptionTimePlus, dls=ts_dls)

# ts features
cat_names = None
cont_names = ts_features_df.columns[:-2]
y_names = 'target'
tab_dls = get_tabular_dls(ts_features_df, cat_names=cat_names, cont_names=cont_names, y_names=y_names, splits=splits)
tab_model = build_tabular_model(TabModel, dls=tab_dls)

# mixed
mixed_dls = get_mixed_dls(ts_dls, tab_dls)
MultiModalNet = MultiInputNet(ts_model, tab_model)
learn = Learner(mixed_dls, MultiModalNet, metrics=[accuracy, RocAuc()])
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,1.794394,1.647609,0.400000,0.803259,00:05


In [ ]:
tab_dls.c, ts_dls.c, ts_dls.cat

(6, 6, True)

In [ ]:
learn.dls.train.one_batch()

((TSTensor(samples:64, vars:24, len:51),
  (tensor([], size=(64, 0), dtype=torch.int64),
   tensor([[ 0.5705,  0.9112,  0.5705,  ..., -1.2493, -0.9131, -1.0031],
           [-0.6821, -0.9772, -0.6821,  ...,  0.3241,  0.0881,  0.8444],
           [ 0.1946,  0.3996,  0.1946,  ..., -1.3933, -0.9556, -0.8486],
           ...,
           [ 0.0433, -0.0804,  0.0433,  ..., -0.3324, -0.4491, -0.2332],
           [-2.0681, -2.0730, -2.0681,  ..., -0.2105, -0.3623, -0.6172],
           [-0.7103, -0.9552, -0.7103,  ..., -1.5821, -0.9989, -1.7734]]))),
 TensorCategory([0, 3, 2, 2, 5, 5, 0, 1, 5, 0, 2, 1, 1, 1, 5, 3, 3, 1, 2, 4, 5, 2, 1, 4,
         3, 1, 4, 3, 2, 4, 3, 4, 1, 0, 0, 5, 4, 3, 1, 2, 5, 5, 4, 1, 5, 0, 3, 2,
         4, 1, 4, 0, 0, 0, 4, 4, 3, 0, 4, 4, 4, 2, 3, 1]))

In [ ]:
learn.loss_func

FlattenedLoss of CrossEntropyLoss()

In [ ]:
#hide
out = create_scripts(); beep(out)